###### Importing Libraries we need

In [1]:
import json
import csv
import requests
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.stats as st
from scipy.stats import linregress
from IPython.display import clear_output
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import statsmodels
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

###### went to https://crashviewer.nhtsa.dot.gov/CrashAPI and got the list of states and built a dictionary off of it

In [2]:
state_id = {1:["Alabama","AL"],
            2:["Alaska","AK"],
            4:["Arizona","AZ"],
            5:["Arkansas","AR"],
            6:["California","CA"],
            8:["Colorado","CO"],
            9:["Conneticut","CT"],
            10:["Delaware", "DE"],
            11:["District of Columbia","DC"],
            12:["Florida","FL"],
            13:["Georgia", "GA"],
            15:["Hawaii","HI"],
            16:["Idaho","ID"],
            17:["Illinois","IL"],
            18:["Indiana","IN"],
            19:["Iowa", "IA"],
            20:["Kansas", "KS"],
            21:["Kentucky","KY"],
            22:["Louisiana","LA"],
            23:["Maine","ME"],
            24:["Maryland","MD"],
            25:["Massachusetts","MA"],
            26:["Michigan","MI"],
            27:["Minnesota","MN"],
            28:["Mississippi","MS"],
            29:["Missouri","MO"],
            30:["Montana","MT"],
            31:["Nebraska","NB"],
            32:["Nevada","NV"],
            33:["New Hampshire","NH"],
            34:["New Jersey","NJ"],
            35:["New Mexico","NM"],
            36:["New York","NY"],
            37:["North Carolina","NC"],
            38:["North Dakota","ND"],
            39:["Ohio","OH"],
            40:["Oklahoma","OK"],
            41:["Oregon","OR"],
            42:["Pennsylvania","PA"],
            43:"Puerto Rico",
            44:["Rhode Island","RI"],
            45:["South Carolina","SC"],
            46:["South Dakota","SD"],
            47:["Tennessee","TN"],
            48:["Texas","TX"],
            49:["Utah","UT"],
            50:["Vermont","VT"],
            51:["Virginia","VA"],
            52:"Virgin Islands",
            53:["Washington", "WA"],
            54:["West Virginia","WV"],
            55:["Wisconsin","WI"],
            56:["Wyoming","WY"]}


###### Create a while loop for user input

In [3]:
state_list=["6","8","12","17","36","39","42","48"]
state_name_list = ["California","Colorado", "Florida", "Illinois", "New York", "Ohio","Pennsylvania","Texas"]
inquiry = str
while inquiry != "done" or inquiry != "" or  (len(state_name_list)) == '10':
    print(f"By default there is\n {state_list} \n {state_name_list}")
    print(len(state_name_list))
    print("there is a limit of 10 states ")
    inquiry = input("Please input the state you are interested in getting Data for by Name or code (ex : CO for Colorado). if you are done type done")
    
    if inquiry == "done":
                    break
    elif len(state_name_list) == 9:
        for i in state_id:
                        if inquiry == state_id[i][0] or inquiry == state_id[i][1]:
                            state_num = i
                            state_list.append(state_num)
                            state_name_list.append(inquiry)
                            print(f"{inquiry} added")
                            clear_output()
        break
        
    else:
                    for i in state_id:
                        if inquiry == state_id[i][0] or inquiry == state_id[i][1]:
                            state_num = i
                            state_list.append(state_num)
                            state_name_list.append(inquiry)
                            print(f"{inquiry} added")
                            clear_output()
                    
print(f"You added {state_name_list} to the list")

By default there is
 ['6', '8', '12', '17', '36', '39', '42', '48'] 
 ['California', 'Colorado', 'Florida', 'Illinois', 'New York', 'Ohio', 'Pennsylvania', 'Texas']
8
there is a limit of 10 states 
Please input the state you are interested in getting Data for by Name or code (ex : CO for Colorado). if you are done type donedone
You added ['California', 'Colorado', 'Florida', 'Illinois', 'New York', 'Ohio', 'Pennsylvania', 'Texas'] to the list


###### Get Results for selected states in JSON format

In [4]:
YearList = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/analytics/GetInjurySeverityCounts?"

crash_data =[]

   
for i,state in enumerate(state_list):
    print(state)
    for k, year in enumerate(YearList):
        
        query_url = (f"{url}fromCaseYear={year}&toCaseYear={year}&state={state}&county=&format=json")
        state_data = requests.get(query_url).json()
        print(year)
        print(state_data)
        caseY = state_data['Results'][0][0]["CaseYear"]
        print(f"this is caseY{caseY}")
        crashC = state_data["Results"][0][0]["CrashCounts"]
        print(f"this is crashC{crashC}")
        totFat = state_data["Results"][0][0]["TotalFatalCounts"]
        print(f"this is totFat{totFat}")
        st_n = state_name_list[i]
        print(st_n)
        #Append the City information into city_data list
        crash_data.append({"State": st_n, "Year":caseY,"Crash Count":crashC,"Total Fatalities": totFat})
        
        
clear_output()            
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------") 


-----------------------------
Data Retrieval Complete      
-----------------------------


###### Build a Dictionary of the data with State Name as  the key 

In [5]:
#predf=dict(zip(state_name_list, Results2))
crash_dict = pd.DataFrame.from_dict(crash_data)
#crash_dict.set_index("State",inplace = True)
# for key in df_dict.keys():
#       pd.DataFrame.from_dict(df_dict, orient='index', index='State')
crash_dict.head()   
crash_dict.dtypes
crash_dict.astype({'Year': 'int64'})
crash_dict.to_csv("Output/FARS_DataFrame.csv",index = None, header=True)

###### Create new DataFrame
in order to save figures you must have some dependencies installed
* conda install -c plotly plotly-orca psutil requests

In [6]:

fig = go.Figure()
merged = crash_dict

#merged['TotalFatalCounts'] = merged['Total Fatalities'].astype(int)
grouped_states = merged.groupby('State')
for states,group in grouped_states['State']:
    year = merged.loc[merged["State"] == states]["Year"]
    crashes = merged.loc[merged["State"] == states]["Total Fatalities"]

    fig.add_trace(go.Scatter(x= year, y=crashes, mode='lines', name=states) )
    fig.update_layout(title='Fatalities By State',
                   xaxis_title='Year',
                   yaxis_title='Fatalities (thousands)')

fig.show()
fig.write_image("Images/fig1.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Git\cmd
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Users\koegs\AppData\Local\Microsoft\WindowsApps
    C:\Users\koegs\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orc

In [7]:
# import MAU data for mobile app adoption, reindex
MAU_data = pd.read_csv('Resources/MAU_data.csv')
MAU_data = MAU_data.fillna(0)
MAU_data_index = MAU_data.set_index('App')
MAU_data_index
MAU_data.to_csv("Output/MAU_Data.csv",index = None, header=True)

In [49]:
# drop years from crash data to ones overlapping with MAU data
grouped_years = merged.groupby('Year')
crashes_per_year = grouped_years["Total Fatalities"].agg('sum')
#crashes_per_year_trimmed = crashes_per_year.drop(['2018','2017','2016','2015'])
# drop years from MAU_data to match years in crash data
MAU_data_trimmed = MAU_data_index.drop(['2002','2003','2004','2005','2006','2007','2008','2009','2019'],axis=1)

# transpose data
MAU_trim_trans = MAU_data_trimmed.transpose()
MAU_trim_trans=MAU_trim_trans.reset_index()
MAU_trim_trans = MAU_trim_trans.rename(columns={"index": "Year"})


In [52]:
year = MAU_trim_trans['Year']
total = MAU_trim_trans['Total']
fig = go.Figure()
fig.add_trace(go.Scatter(x=year, y=total, fill='tozeroy'))
fig.update_layout(title='Fatal Crashes vs MAU of Top Mobile Apps in the US, 2010-2018 '+ line_eq,
                   xaxis_title="MAU's",
                   yaxis_title='Fatalities (thousands)')
fig.update_layout(title='Monthly Active Users of the top 20 Mobile Apps',
                   xaxis_title="Year",
                   yaxis_title='MAUs')
fig.show()
fig.write_image("images/MAUs.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Git\cmd
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Users\koegs\AppData\Local\Microsoft\WindowsApps
    C:\Users\koegs\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orc

In [42]:
# set x and y values for scatter plot
x_values = MAU_trim_total
y_values = crashes_per_year

fig = go.Figure()

# Add traces

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
fig=px.scatter(x=x_values, y=y_values,trendline='ols')
fig.update_layout(title='Fatal Crashes vs MAU of Top Mobile Apps in the US, 2010-2018 ',
                   xaxis_title="MAU's",
                   yaxis_title='Fatalities (thousands)')

fig.show()

fig.write_image("images/fig2.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Git\cmd
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Users\koegs\AppData\Local\Microsoft\WindowsApps
    C:\Users\koegs\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orc

In [10]:
# import and read smartphone adoption datafile
phone_adoption_df = pd.read_csv('Resources\smartphone adoption.csv')

#trim years to match crash data years
phone_adoption_trim = phone_adoption_df.drop([9,10,11,12,13])
phone_adoption_df.to_csv("Output/phone_adoption_df.csv",index = None, header=True)

In [51]:
# plot total fatal crashes by state over time
fig = go.Figure()
year = phone_adoption_trim['year']
smart_users = phone_adoption_trim["number of smartphone users in US (millions)"]

fig = px.bar(
    phone_adoption_trim, 
    x=year, 
    y=smart_users, 
    text=smart_users, 
    height=600)
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(
    title="The Number of Smartphone Users in the US (Millions)",
    xaxis_title = "Year",
    yaxis_title=""
        )
fig.show()
fig.write_image("images/phone_users.png")


ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Git\cmd
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Users\koegs\AppData\Local\Microsoft\WindowsApps
    C:\Users\koegs\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orc

In [45]:
x_values2 = phone_adoption_trim['number of smartphone users in US (millions)']
y_values = crashes_per_year

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values2, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
fig=px.scatter(x=x_values, y=y_values,trendline='ols')

fig.update_layout(title='Fatal Crashes vs Number of Smartphone Users in the US, 2010-2018 ' + line_eq,
                   xaxis_title="Number of Smartphone users in the US",
                   yaxis_title='Fatalities Crashes')
fig.show()

fig.write_image("images/fig3.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Git\cmd
    C:\Users\koegs\Anaconda3
    C:\Users\koegs\Anaconda3\Library\mingw-w64\bin
    C:\Users\koegs\Anaconda3\Library\usr\bin
    C:\Users\koegs\Anaconda3\Library\bin
    C:\Users\koegs\Anaconda3\Scripts
    C:\Users\koegs\AppData\Local\Microsoft\WindowsApps
    C:\Users\koegs\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orc